In [1]:
import pandas as pd
import numpy as np

## Training

In [2]:
train_data = pd.read_csv("data/coding5_train.csv").to_numpy()

lam = 1
T = 20
p = train_data.shape[1] - 1

beta = np.zeros(p) # slope
alpha = 0          # intercept
t = 0

np.random.seed(1234)

for t in range(T):

    np.random.shuffle(train_data) 
    
    for row in enumerate(train_data):
        x = row[1][:-1]
        y = 1 if row[1][-1] == 6 else -1 # map 5 to -1 and 6 to 1
        
        t += 1
        ita = (1 / (t * lam))
        
        # calculate gradients
        slope_gradient = (lam * beta)
        intercept_gradient = 0
        
        if y * ((x @ beta) + alpha) < 1:
            slope_gradient -= (y * x)
            intercept_gradient -= y    

        # update coefficients
        beta -= ita * slope_gradient
        alpha -= ita * intercept_gradient

## Testing

In [8]:
test_data = pd.read_csv("data/coding5_test.csv")
X_test = test_data.drop(columns=["Y"]).to_numpy()

Y_test = test_data["Y"].to_numpy()
Y_test = np.where(Y_test == 6, 1, -1) # map 5 to -1 and 6 to 1

test_predictions = np.where(((X_test @ beta) + alpha) > 0, 1, -1)

correct = np.where(test_predictions == Y_test, 1, 0)
print("Test error rate:",  1 - (np.sum(correct) / Y_test.shape[0]))

Test error rate: 0.036666666666666625


In [9]:
def pretty_print_matrix(confusion_matrix, is_test=False):
    title = "Test Confusion Matrix" if is_test else "Train Confusion Matrix"

    space = "    "
    print(f"========{title}========")
    print(f"{space}{space}{space}{space}Actual")
    # print(f"{space}Positive{space}Negative")
    print(f"{space}{space}{space}{confusion_matrix[0][0]}{space}{space}{confusion_matrix[0][1]}")
    print(f"Predicted")
    print(f"{space}{space}{space}{confusion_matrix[1][0]}{space}{space}{confusion_matrix[1][1]}")

    bottom_border = ['=' for i in range(len(title) + 16)]
    print(''.join(bottom_border))

In [11]:
test_confusion = [[0, 0], [0, 0]]

for pred, actual in zip(test_predictions, Y_test):
    if pred == 1 and actual == 1:
        test_confusion[0][0] += 1
    elif pred == 1 and actual == -1:
        test_confusion[0][1] += 1
    elif pred == -1 and actual == 1:
        test_confusion[1][0] += 1
    else:
        test_confusion[1][1] += 1
        
pretty_print_matrix(test_confusion, is_test=True)

train_confusion = [[0, 0], [0, 0]]
X_train = train_data[:, :-1]
train_predictions = np.where(((X_train @ beta) + alpha) > 0, 1, -1)

Y_train = np.where(train_data[:, -1] == 6, 1, -1) # map 5 to -1 and 6 to 1

for pred, actual in zip(train_predictions, Y_train):
    if pred == 1 and actual == 1:
        train_confusion[0][0] += 1
    elif pred == 1 and actual == -1:
        train_confusion[0][1] += 1
    elif pred == -1 and actual == 1:
        train_confusion[1][0] += 1
    else:
        train_confusion[1][1] += 1

pretty_print_matrix(train_confusion)

========Test Confusion Matrix========
                Actual
            285        7
Predicted
            15        293
========Train Confusion Matrix========
                Actual
            98        0
Predicted
            2        100
